# **3278. Find Candidates for Data Scientist Position II**

**Table: Candidates**
``` sql
+--------------+---------+ 
| Column Name  | Type    | 
+--------------+---------+ 
| candidate_id | int     | 
| skill        | varchar |
| proficiency  | int     |
+--------------+---------+
```
(candidate_id, skill) is the unique key for this table.

Each row includes candidate_id, skill, and proficiency level (1-5).

**Table: Projects**
``` sql
+--------------+---------+ 
| Column Name  | Type    | 
+--------------+---------+ 
| project_id   | int     | 
| skill        | varchar |
| importance   | int     |
+--------------+---------+
```
(project_id, skill) is the primary key for this table.

Each row includes project_id, required skill, and its importance (1-5) for the project.

Leetcode is staffing for multiple data science projects. Write a solution to find the best candidate for 

each project based on the following criteria:

Candidates must have all the skills required for a project.

Calculate a score for each candidate-project pair as follows:

Start with 100 points

Add 10 points for each skill where proficiency > importance

Subtract 5 points for each skill where proficiency < importance

Include only the top candidate (highest score) for each project. If there’s a tie, choose the candidate with the lower candidate_id. If there is no suitable candidate for a project, do not return that project.

Return a result table ordered by project_id in ascending order.

The result format is in the following example.

 

**Example:**

**Input:**

**Candidates table:**
``` sql
+--------------+-----------+-------------+
| candidate_id | skill     | proficiency |
+--------------+-----------+-------------+
| 101          | Python    | 5           |
| 101          | Tableau   | 3           |
| 101          | PostgreSQL| 4           |
| 101          | TensorFlow| 2           |
| 102          | Python    | 4           |
| 102          | Tableau   | 5           |
| 102          | PostgreSQL| 4           |
| 102          | R         | 4           |
| 103          | Python    | 3           |
| 103          | Tableau   | 5           |
| 103          | PostgreSQL| 5           |
| 103          | Spark     | 4           |
+--------------+-----------+-------------+
```
**Projects table:**
``` sql
+-------------+-----------+------------+
| project_id  | skill     | importance |
+-------------+-----------+------------+
| 501         | Python    | 4          |
| 501         | Tableau   | 3          |
| 501         | PostgreSQL| 5          |
| 502         | Python    | 3          |
| 502         | Tableau   | 4          |
| 502         | R         | 2          |
+-------------+-----------+------------+
```
**Output:**
``` sql
+-------------+--------------+-------+
| project_id  | candidate_id | score |
+-------------+--------------+-------+
| 501         | 101          | 105   |
| 502         | 102          | 130   |
+-------------+--------------+-------+
```
**Explanation:**

For Project 501, Candidate 101 has the highest score of 105. All other candidates have the same score but Candidate 101 has the lowest candidate_id among them.

For Project 502, Candidate 102 has the highest score of 130.

The output table is ordered by project_id in ascending order.

**Solution-1:**

``` sql
with cte as(
select c.candidate_id, p.project_id,
100 + SUM(case when proficiency > importance then 10  when proficiency < importance then -5
else 0 end) as score,
RANK()over(partition by p.project_id order by 100 + SUM(case when proficiency > importance then 10  when proficiency < importance then -5 else 0 end) desc, c.candidate_id ASC) as rnk,
(select count(*) from projects pp where pp.project_id = p.project_id) as project_skills_count,
count(c.skill) as cand_skills_for_project_count
from candidates c, projects p
where c.skill = p.skill
group by 1,2
having project_skills_count = cand_skills_for_project_count)

select project_id, candidate_id, score
from cte where rnk = 1 
order by project_id ASC

**Solution-2:**

``` sql
with cte as(
    select p.project_id, candidate_id, 
        sum(case when proficiency > importance then 10
                 when proficiency = importance then 0
                 else -5
            end) + 100 as score,
        count(*) as cnt_skill
    from Projects p
    left join Candidates c
    on p.skill = c.skill
    group by 1, 2
),

-- criteria 1 (where clause) + 2 (row_number() window funtion):
tmp as(
    select project_id, candidate_id, score,
        row_number()over(partition by project_id order by score desc, candidate_id asc) as rn
    from cte
    where (project_id, cnt_skill) in (
        select project_id, count(*)
        from Projects
        group by 1
    )
)

select project_id, candidate_id, score
from tmp
where rn = 1
order by 1

**Solution-3:**

``` sql
with tmp1 as 
(
    select   p.project_id
            ,c.candidate_id
            ,c.skill
            ,p.importance
            ,c.proficiency
            ,100 + sum(case when c.proficiency > p.importance then 10
                            when c.proficiency < p.importance then -5
                            else 0 end) over(partition by p.project_id, c.candidate_id) as score
            ,(select count(*) from projects pp where pp.project_id = p.project_id) as project_skills_count
            ,count(c.skill) over(partition by p.project_id, c.candidate_id) as cand_skills_for_project_count
    from projects p
        left join candidates c on p.skill = c.skill
),tmp2 as
(
    select   distinct
             project_id
            ,candidate_id
            ,score
            ,row_number() over(partition by project_id order by score desc, candidate_id asc) as rnk
    from tmp1 
    where project_skills_count = cand_skills_for_project_count
)
select   project_id
        ,candidate_id
        ,score
from tmp2
where rnk = 1
order by project_id asc